# Nena Parser 3
## Pure Python Parser for NENA Texts

In [1]:
import re
import collections

In [2]:
import re
import collections
from pathlib import Path
import json
import unicodedata

VERSION = '2.0'
PROJECT = Path.home().joinpath('github/CambridgeSemiticsLab/nena_corpus')
STANDARDS = PROJECT.joinpath('standards')
LEXICONS = STANDARDS.joinpath('lexicons')
TEXT_INPUT = PROJECT.joinpath(f'texts/{VERSION}')
JSON_OUTPUT = PROJECT.joinpath(f'parsed_texts/{VERSION}')

# prepare alphabet and punctuation standards for processing
alphabet_std = STANDARDS.joinpath('alphabet.json')
punctuation_std = STANDARDS.joinpath('punctuation.json')
lang_std = STANDARDS.joinpath('foreign_languages.json')

with open(alphabet_std, 'r') as infile:
    alphabet_data = json.load(infile)    
with open(punctuation_std, 'r') as infile:    
    punct_data = json.load(infile)
with open(lang_std, 'r') as infile:
    lang_data = json.load(infile)

alphabet_re = '|'.join(let['decomposed_regex'] for let in alphabet_data)
punct_begin_re = '|'.join(punct['decomposed_regex'] for punct in punct_data
                            if punct['position'] == 'begin')

punct_end_re = '|'.join(punct['decomposed_regex'] for punct in punct_data 
                            if punct['position'] == 'end')

foreign_codes = '|'.join(lang['code'] for lang in lang_data)
                               
# prepare letter and punctuation data for 
# matching on a letter-by-letter basis
metakeys = {
    'punctuation': {'decomposed_string', 'class', 'position', 'modifies'},
    'letter': {'decomposed_string', 'class', 'point', 'manner', 'phonation'},
}
char_metadata = collections.defaultdict(list)
for item, data in [('letter', alphabet_data), ('punctuation', punct_data)]:
    for chardata in data:
        re_pattern = re.compile(chardata['decomposed_regex'])
        metadata = {k:v for k,v in chardata.items() if k in metakeys[item]}
        char_metadata[item].append((re_pattern, metadata))
        
        
# prepare lexicon matching by dialect
dialect2lexicon = collections.defaultdict()
dialect2inflects = collections.defaultdict()

# load lexicon and inflection data by dialect
for dialect_dir in LEXICONS.glob('*'):
    
    # check paths and existence of relevant data
    lexicon_file = dialect_dir.joinpath('lexicon.json')
    inflects_file = dialect_dir.joinpath('inflections.json')
    if not (lexicon_file.exists() and inflects_file.exists()):
        continue
        
    # load and save files by dialect in dirs
    dialect = dialect_dir.name
    with open(lexicon_file, 'r') as infile:
        dialect2lexicon[dialect] = json.load(infile)
    with open(inflects_file, 'r') as infile:
        dialect2inflects[dialect] = json.load(infile)
        
# exclude entry text from lexeme data for now
# as it's too long; also exclude form string
for dialect, lexs in dialect2lexicon.items():
    for lex, ldata in lexs.items():
        lexs[lex] = {
            k:v for k,v in ldata.items()
                if k not in {'entry', 'form'}
        }
        
# compile regexes for lexicon matching
for dialect, inflects in dialect2inflects.items():
    for inflect in inflects:
        inflect['form'] = re.compile(re.escape(inflect['form']))
    
def normalize_nena(word):
    """Strip vowel accents from NENA string."""
    accents = '\u0300|\u0301|\u0304|\u0306|\u0308|\u0303'
    norm = unicodedata.normalize('NFD', word) # decompose for accent stripping
    norm = re.sub(accents, '', norm) # strip accents
    return norm
    
def parse_word(word, dialect):
    """Match a word with appropriate parsing data from lexicon."""
    
    if dialect not in dialect2inflects:
        return []
    
    word = normalize_nena(word)
    
    # find all matching surface forms
    matches = []
    for i, form in enumerate(dialect2inflects[dialect]):
        if form['form'].fullmatch(word):
            # filter out 'form' from data
            form_data = {
                k:v for k,v in form.items()
                    if k != 'form'
            }
            matches.append(form_data)
    
    # retrieve default lexicon data
    parsings = []
    for match in matches:
        lex_str = match['lex']
        lex_data = dialect2lexicon[dialect][lex_str]
        lex_data.update(match)
        parsings.append(lex_data)
        
    return parsings

In [61]:
sent_puncts = '|'.join(
    p['decomposed_regex'] for p in punct_data
        if p['modifies'] in {'sentence', 'subsentence'}
        and p['class'] == 'separator'
        and p['decomposed_regex'] != ':'
)


sent_puncts

',|\\.\\.\\.|—|;|(?<!\\.)\\.(?!\\.)|\\?|!'

In [58]:
punct_data

[{'decomposed_regex': '⁺',
  'decomposed_string': '⁺',
  'class': 'phonetic',
  'position': 'begin',
  'modifies': 'stress group',
  'codepoints': [8314]},
 {'decomposed_regex': '(?<= )"',
  'decomposed_string': '"',
  'class': 'separator',
  'modifies': 'text',
  'position': 'begin',
  'codepoints': [34]},
 {'decomposed_regex': ' ',
  'decomposed_string': ' ',
  'class': 'separator',
  'modifies': 'word',
  'position': 'end',
  'codepoints': [32]},
 {'decomposed_regex': '-',
  'decomposed_string': '-',
  'class': 'connector',
  'modifies': 'stress group',
  'position': 'end',
  'codepoints': [45]},
 {'decomposed_regex': '=',
  'decomposed_string': '=',
  'class': 'connector',
  'modifies': 'stress group',
  'position': 'end',
  'codepoints': [61]},
 {'decomposed_regex': 'ˈ',
  'decomposed_string': 'ˈ',
  'class': 'separator',
  'modifies': 'intonation group',
  'position': 'end',
  'codepoints': [712]},
 {'decomposed_regex': ',',
  'decomposed_string': ',',
  'class': 'separator',
  '

## Example Text

Below is a dummy text we can use to test the parsers on, written in the
[NENA Markup](../docs/nena_format.md) standard.

In [70]:
example = unicodedata.normalize('NFD', '''
dialect:: Urmi_C
title:: When Shall I Die?
encoding:: UTF8
speakers:: YD=Yulia Davudi, GK=Geoffrey Khan, CK=Cody Kingham
place:: +Hassar +Baba-čanɟa, N
transcriber:: Geoffrey Khan
text_id:: A32 

(1 0:00) <E:Ok> xá-yuma ⁺malla ⁺Nasrádən váyələ tíva ⁺ʾal-k̭èsa.ˈ xá mən-nášə ⁺vàrəva,ˈ mə́rrə
 ⁺màllaˈ ʾátən ʾo-k̭ésa pràmut,ˈ bət-nàplət.ˈ mə́rrə <P:bŏ́ro> bàbaˈ ʾàtən=daˈ
 ⁺šúla lə̀tluxˈ tíyyət b-dìyyi k̭ítət.ˈ ⁺šúk̭ si-⁺bar-⁺šùlux.ˈ ʾána ⁺šūl-ɟànilə.ˈ
náplən nàplən.ˈ (2 0:08) ⁺hàlaˈ ʾo-náša léva xíša xá ⁺ʾəsrá ⁺pasulyày,ˈ ⁺málla
bitáyələ drúm ⁺ʾal-⁺ʾàrra.ˈ bək̭yámələ ⁺bərxáṱələ ⁺bàru.ˈ màraˈ ⁺maxlèta,ˈ ʾátən
 ⁺dílux ʾána bət-náplənva m-⁺al-ʾilàna.ˈ bas-tánili xázən ʾána ʾíman bət-mètən.ˈ
ʾo-náša xzílə k̭at-ʾá ⁺màllaˈ hónu xáč̭č̭a ... ⁺basùrələˈ mə́rrə k̭àtuˈ ⁺maxlèta,ˈ
mə̀drə,ˈ (GK) maxlèta? (YD) ⁺rába ⁺maxlèta.ˈ mə́rrə k̭at-ʾíman xmártux ⁺ṱlá ɟáhə 
⁺ʾarṱàla,ˈ ʾó-yuma mètət.ˈ ʾó-yumət xmártux ⁺ṱlá ɟáhə ⁺ʾarṱàla,ˈ ʾó-yuma mètət.ˈ 

(3 0:14) ⁺málla múttəva ... ⁺ṱànaˈ ⁺yak̭úyra ⁺ʾal-xmàrta.ˈ ⁺ṱànaˈ mə́ndi ⁺rába múttəva 
⁺ʾal-xmàrtaˈ ʾu-xmàrtaˈ ⁺báyyava ʾask̭áva ⁺ʾùllul.ˈ ʾu-bas-pòxa ⁺plə́ṱlə mənnó.ˈ ṱə̀r,ˈ
 ⁺riṱàla.ˈ ⁺málla mə́rrə ʾàha,ˈ ʾána dū́n k̭arbúnə k̭a-myàta.ˈ (4 0:19) xáč̭č̭a=da sə̀k̭laˈ
xa-xìta.ˈ (CK)<E:Hello this is a test> (YD)ɟánu mudməxxálə ⁺ʾal-⁺ʾàrra.ˈ mə̀rrəˈ 
xína ⁺dā́n mòtila.ˈ ʾē=t-d-⁺ṱlàˈ ⁺málla mə̀tlə.ˈ nàšə,ˈ xuyravàtuˈ xə́šlun tílun mə̀rrunˈ 
ʾa mù-vadət? k̭a-mú=ivət ⁺tàmma?ˈ mə́rrə xob-ʾána mìtən.ˈ lá bəxzáyətun k̭at-mìtən!ˈ lá mə́rrun 
ʾat-xàya!ˈ hamzùməvət.ˈ bəšvák̭una ⁺tàmaˈ màraˈ xmàrələ,ˈ lélə ⁺p̭armùyə.ˈ
 ''')

## Parser

In [39]:
class NenaParser:
    
    def __init__(self):
        # compile primary regex patterns
        self.blocks = re.compile(r'\n\n')
        
    def parse(self, text, dialect):
        """Parse a NENA format text into structured, analyzed linguistic data.
        
        Args:
            text: a string containing NENA Markup-formatted text.
        
        Returns:
            List of structured, parsed data.
        """
        metadata, text_block = self.get_blocks(text)
        metadata_parsed = self.parse_metablock(metadata)
        return metadata_parsed
        
    def get_blocks(self, text):
        """Split text into metadata and text blocks.
        
        Args:
            text: a NENA markup string consisting of 
                a metadata block separated from a text block
                by two newlines.
                
        Returns:
            two-tuple of (metadata block, text block) strings
        """
        return self.blocks.split(text, 1)

    def parse_metablock(self, metablock):
        """Convert metadata text block into dictionary.

        Args: 
            metablock: a multi-line string where each
                line corresponds to a single metadata entry
                for the text; e.g.
                ```
                dialect:: Barwar
                title:: A Hundred Gold Coins
                ```
        Returns:
            dictionary of metadata key, value pairs.
        """
        metablock = metablock.strip()
        attribs = metablock.split('\n')
        metadata = dict(self.parse_meta_attrib(entry) for entry in attribs)
        return metadata

    def parse_meta_attrib(self, meta_line):
        """Parse a line of metadata into a key, value pair.

        Args:
            meta_line: a string of metadata corresponding to
                a single line in metadata block
                e.g. "dialect:: Barwar"
        Returns:
            two-tuple of (key, value) for the attribute.
        """
        key, value = meta_line.split('::')
        key, value = key.strip(), value.strip()
        if key == 'speakers':
            value = self.parse_speakers_attrib(value)
        return (key, value)
    
    def parse_speakers_attrib(self, speaker_line):
        """Parse a speaker attribute line into a dictionary.

        Args: 
            speaker_line: a string containing speaker nickname assignments
                e.g. "GK=Geoffrey Khan, CK=Cody Kingham"

        Returns:
            dict where each key is an initial and value is speaker's 
            name as a string.
        """
        speakers = {}
        for speakset in speaker_line.split(','):
            initials, speaker = speakset.split('=')
            initials, speaker = initials.strip(), speaker.strip()
            speakers[initials] = speaker
        return speakers

In [40]:
parser = NenaParser()

parser.parse(example, 'Urmi_C')

{'dialect': 'Urmi_C',
 'title': 'When Shall I Die?',
 'encoding': 'UTF8',
 'speakers': {'YD': 'Yulia Davudi',
  'GK': 'Geoffrey Khan',
  'CK': 'Cody Kingham'},
 'place': '+Hassar +Baba-čanɟa, N',
 'transcriber': 'Geoffrey Khan',
 'text_id': 'A32'}

In [75]:
test_text = '''
(1 0:00) <E:Ok> xá-yuma ⁺malla ⁺Nasrádən váyələ tíva ⁺ʾal-k̭èsa.ˈ xá mən-nášə ⁺vàrəva,ˈ mə́rrə
 ⁺màllaˈ ʾátən ʾo-k̭ésa pràmut,ˈ bət-nàplət.ˈ mə́rrə <P:bŏ́ro> bàbaˈ ʾàtən=daˈ
 ⁺šúla lə̀tluxˈ tíyyət b-dìyyi k̭ítət.ˈ ⁺šúk̭ si-⁺bar-⁺šùlux.ˈ ʾána ⁺šūl-ɟànilə.ˈ
náplən nàplən.ˈ (2 0:08) ⁺hàlaˈ ʾo-náša léva xíša xá ⁺ʾəsrá ⁺pasulyày,ˈ ⁺málla
bitáyələ drúm ⁺ʾal-⁺ʾàrra.ˈ bək̭yámələ ⁺bərxáṱələ ⁺bàru.ˈ màraˈ ⁺maxlèta,ˈ ʾátən
 ⁺dílux ʾána bət-náplənva m-⁺al-ʾilàna.ˈ bas-tánili xázən ʾána ʾíman bət-mètən.ˈ
ʾo-náša xzílə k̭at-ʾá ⁺màllaˈ hónu xáč̭č̭a ... ⁺basùrələˈ mə́rrə k̭àtuˈ ⁺maxlèta,ˈ
mə̀drə,ˈ (GK) maxlèta? (YD) ⁺rába ⁺maxlèta.ˈ mə́rrə k̭at-ʾíman xmártux ⁺ṱlá ɟáhə 
⁺ʾarṱàla,ˈ ʾó-yuma mètət.ˈ ʾó-yumət xmártux ⁺ṱlá ɟáhə ⁺ʾarṱàla,ˈ ʾó-yuma mètət.ˈ 

(3 0:14) ⁺málla múttəva ... ⁺ṱànaˈ ⁺yak̭úyra ⁺ʾal-xmàrta.ˈ ⁺ṱànaˈ mə́ndi ⁺rába múttəva 
⁺ʾal-xmàrtaˈ ʾu-xmàrtaˈ ⁺báyyava ʾask̭áva ⁺ʾùllul.ˈ ʾu-bas-pòxa ⁺plə́ṱlə mənnó.ˈ ṱə̀r,ˈ
 ⁺riṱàla.ˈ ⁺málla mə́rrə ʾàha,ˈ ʾána dū́n k̭arbúnə k̭a-myàta.ˈ (4 0:19) xáč̭č̭a=da sə̀k̭laˈ
xa-xìta.ˈ (CK)<E:Hello this is a test> (YD)ɟánu mudməxxálə ⁺ʾal-⁺ʾàrra.ˈ mə̀rrəˈ 
xína ⁺dā́n mòtila.ˈ ʾē=t-d-⁺ṱlàˈ ⁺málla mə̀tlə.ˈ nàšə,ˈ xuyravàtuˈ xə́šlun tílun mə̀rrunˈ 
ʾa mù-vadət? k̭a-mú=ivət ⁺tàmma?ˈ mə́rrə xob-ʾána mìtən.ˈ lá bəxzáyətun k̭at-mìtən!ˈ lá mə́rrun 
ʾat-xàya!ˈ hamzùməvət.ˈ bəšvák̭una ⁺tàmaˈ màraˈ xmàrələ,ˈ lélə ⁺p̭armùyə.ˈ
'''

test_paragraph = '''\
(1 0:00) <E:Ok> xá-yuma ⁺malla ⁺Nasrádən váyələ tíva ⁺ʾal-k̭èsa.ˈ xá mən-nášə ⁺vàrəva,ˈ mə́rrə
 ⁺màllaˈ ʾátən ʾo-k̭ésa pràmut,ˈ bət-nàplət.ˈ mə́rrə <P:bŏ́ro> bàbaˈ ʾàtən=daˈ
 ⁺šúla lə̀tluxˈ tíyyət b-dìyyi k̭ítət.ˈ ⁺šúk̭ si-⁺bar-⁺šùlux.ˈ ʾána ⁺šūl-ɟànilə.ˈ
náplən nàplən.ˈ (2 0:08) ⁺hàlaˈ ʾo-náša léva xíša xá ⁺ʾəsrá ⁺pasulyày,ˈ ⁺málla
bitáyələ drúm ⁺ʾal-⁺ʾàrra.ˈ bək̭yámələ ⁺bərxáṱələ ⁺bàru.ˈ màraˈ ⁺maxlèta,ˈ ʾátən
 ⁺dílux ʾána bət-náplənva m-⁺al-ʾilàna.ˈ bas-tánili xázən ʾána ʾíman bət-mètən.ˈ
ʾo-náša xzílə k̭at-ʾá ⁺màllaˈ hónu xáč̭č̭a ... ⁺basùrələˈ mə́rrə k̭àtuˈ ⁺maxlèta,ˈ
mə̀drə,ˈ (GK) maxlèta? (YD) ⁺rába ⁺maxlèta. mə́rrə k̭at-ʾíman xmártux ⁺ṱlá ɟáhə 
⁺ʾarṱàla,ˈ ʾó-yuma mètət.ˈ ʾó-yumət xmártux ⁺ṱlá ɟáhə ⁺ʾarṱàla,ˈ ʾó-yuma mètət.ˈ\
'''

In [ ]:
def parse_textblock(textblock):
    """Parse a text block into structured data.
    
    Args:
        textblock: a text block in NENA markup format.
        
    Returns:
        List of metadata with parsings.
    """
    paragraphs = split_paragraphs(textblock)
    
    
    
    sentences = split_sentences(paragraphs)
    
def split_paragraphs(textblock):
    """Split text into paragraphs.
    
    Args:
        textblock: string with NENA markup text where paragraphs
            are separated by double newlines.
    
    Returns:
        list of strings where each string is a paragraph.
    """
    return textblock.split('\n\n')

def split_sentences(paragraph):
    """Splits off sentences based on punctuation.
    
    Args:
        paragraphs: list of strings where each string corresponds
            with a paragraph.
            
    Returns:
        list of strings which correspond with sentences
    """
    sent_mark = 
    return 

In [45]:
re.split(test_paragraph)

(1 0:00) <E:Ok> xá-yuma ⁺malla ⁺Nasrádən váyələ tíva ⁺ʾal-k̭èsa.ˈ xá mən-nášə ⁺vàrəva,ˈ mə́rrə
 ⁺màllaˈ ʾátən ʾo-k̭ésa pràmut,ˈ bət-nàplət.ˈ mə́rrə <P:bŏ́ro> bàbaˈ ʾàtən=daˈ
 ⁺šúla lə̀tluxˈ tíyyət b-dìyyi k̭ítət.ˈ ⁺šúk̭ si-⁺bar-⁺šùlux.ˈ ʾána ⁺šūl-ɟànilə.ˈ
náplən nàplən.ˈ (2 0:08) ⁺hàlaˈ ʾo-náša léva xíša xá ⁺ʾəsrá ⁺pasulyày,ˈ ⁺málla
bitáyələ drúm ⁺ʾal-⁺ʾàrra.ˈ bək̭yámələ ⁺bərxáṱələ ⁺bàru.ˈ màraˈ ⁺maxlèta,ˈ ʾátən
 ⁺dílux ʾána bət-náplənva m-⁺al-ʾilàna.ˈ bas-tánili xázən ʾána ʾíman bət-mètən.ˈ
ʾo-náša xzílə k̭at-ʾá ⁺màllaˈ hónu xáč̭č̭a ... ⁺basùrələˈ mə́rrə k̭àtuˈ ⁺maxlèta,ˈ
mə̀drə,ˈ (GK) maxlèta? (YD) ⁺rába ⁺maxlèta.ˈ mə́rrə k̭at-ʾíman xmártux ⁺ṱlá ɟáhə 
⁺ʾarṱàla,ˈ ʾó-yuma mètət.ˈ ʾó-yumət xmártux ⁺ṱlá ɟáhə ⁺ʾarṱàla,ˈ ʾó-yuma mètət.ˈ


In [77]:
[''.join(sub_sent) for sub_sent in re.findall(f'(.*?)({sent_puncts})(ˈ)?', test_paragraph.replace('\n', ' '))]

['(1 0:00) <E:Ok> xá-yuma ⁺malla ⁺Nasrádən váyələ tíva ⁺ʾal-k̭èsa.ˈ',
 ' xá mən-nášə ⁺vàrəva,ˈ',
 ' mə́rrə  ⁺màllaˈ ʾátən ʾo-k̭ésa pràmut,ˈ',
 ' bət-nàplət.ˈ',
 ' mə́rrə <P:bŏ́ro> bàbaˈ ʾàtən=daˈ  ⁺šúla lə̀tluxˈ tíyyət b-dìyyi k̭ítət.ˈ',
 ' ⁺šúk̭ si-⁺bar-⁺šùlux.ˈ',
 ' ʾána ⁺šūl-ɟànilə.ˈ',
 ' náplən nàplən.ˈ',
 ' (2 0:08) ⁺hàlaˈ ʾo-náša léva xíša xá ⁺ʾəsrá ⁺pasulyày,ˈ',
 ' ⁺málla bitáyələ drúm ⁺ʾal-⁺ʾàrra.ˈ',
 ' bək̭yámələ ⁺bərxáṱələ ⁺bàru.ˈ',
 ' màraˈ ⁺maxlèta,ˈ',
 ' ʾátən  ⁺dílux ʾána bət-náplənva m-⁺al-ʾilàna.ˈ',
 ' bas-tánili xázən ʾána ʾíman bət-mètən.ˈ',
 ' ʾo-náša xzílə k̭at-ʾá ⁺màllaˈ hónu xáč̭č̭a ...',
 ' ⁺basùrələˈ mə́rrə k̭àtuˈ ⁺maxlèta,ˈ',
 ' mə̀drə,ˈ',
 ' (GK) maxlèta?',
 ' (YD) ⁺rába ⁺maxlèta.',
 ' mə́rrə k̭at-ʾíman xmártux ⁺ṱlá ɟáhə  ⁺ʾarṱàla,ˈ',
 ' ʾó-yuma mètət.ˈ',
 ' ʾó-yumət xmártux ⁺ṱlá ɟáhə ⁺ʾarṱàla,ˈ',
 ' ʾó-yuma mètət.ˈ']

In [80]:
re.findall(f'.*?(?<!\.)\.(?!\.)ˈ?', test_paragraph.replace('\n', ' '))

['(1 0:00) <E:Ok> xá-yuma ⁺malla ⁺Nasrádən váyələ tíva ⁺ʾal-k̭èsa.ˈ',
 ' xá mən-nášə ⁺vàrəva,ˈ mə́rrə  ⁺màllaˈ ʾátən ʾo-k̭ésa pràmut,ˈ bət-nàplət.ˈ',
 ' mə́rrə <P:bŏ́ro> bàbaˈ ʾàtən=daˈ  ⁺šúla lə̀tluxˈ tíyyət b-dìyyi k̭ítət.ˈ',
 ' ⁺šúk̭ si-⁺bar-⁺šùlux.ˈ',
 ' ʾána ⁺šūl-ɟànilə.ˈ',
 ' náplən nàplən.ˈ',
 ' (2 0:08) ⁺hàlaˈ ʾo-náša léva xíša xá ⁺ʾəsrá ⁺pasulyày,ˈ ⁺málla bitáyələ drúm ⁺ʾal-⁺ʾàrra.ˈ',
 ' bək̭yámələ ⁺bərxáṱələ ⁺bàru.ˈ',
 ' màraˈ ⁺maxlèta,ˈ ʾátən  ⁺dílux ʾána bət-náplənva m-⁺al-ʾilàna.ˈ',
 ' bas-tánili xázən ʾána ʾíman bət-mètən.ˈ',
 ' ʾo-náša xzílə k̭at-ʾá ⁺màllaˈ hónu xáč̭č̭a ... ⁺basùrələˈ mə́rrə k̭àtuˈ ⁺maxlèta,ˈ mə̀drə,ˈ (GK) maxlèta? (YD) ⁺rába ⁺maxlèta.',
 ' mə́rrə k̭at-ʾíman xmártux ⁺ṱlá ɟáhə  ⁺ʾarṱàla,ˈ ʾó-yuma mètət.ˈ',
 ' ʾó-yumət xmártux ⁺ṱlá ɟáhə ⁺ʾarṱàla,ˈ ʾó-yuma mètət.ˈ']